In [1]:
import pandas as pd
pd.options.display.max_rows = 100

In [2]:
files = {
    'diet_example': 'https://regepi.bwh.harvard.edu/health/Oxalate/files/Low%20Oxalate%20Diet%20Example.xls',
    'oxalate_content' : 'https://regepi.bwh.harvard.edu/health/Oxalate/files/Oxalate%20Content%20of%20Foods.xls',
    'table_of_foods' : 'https://regepi.bwh.harvard.edu/health/Oxalate/files/Oxalate%20Table%20of%20Foods.xls'
}

In [3]:
raw_data = pd.read_excel(files['oxalate_content'])

In [4]:
df = raw_data.copy()

# Filter Data

In [5]:
df = df[['Food Group', 'Food Item', 'Serving size','OxalateCategory','Oxalate Value']]

In [6]:
#get rid of rows with null Food Item's
df = df[df['Food Item'].notnull()]

In [7]:
#forward fill Food Groups
df['Food Group'] = df['Food Group'].fillna(method='ffill')

In [8]:
#removes the "mg" suffix if applicable
def remove_mg(value):
    if value.endswith('mg'):
        return value[:-2]


In [9]:
#Apply removal of "mg" suffix
df['Oxalate Value'] = df['Oxalate Value'].apply(remove_mg)

#convert to numeric data type
df['Oxalate Value'] = df['Oxalate Value'].apply(pd.to_numeric)

In [10]:
df.describe()

,Oxalate Value
count,536.000000
mean,15.046642
std,52.923536
min,0.000000
25%,1.000000
50%,4.000000
75%,13.000000
max,755.000000


In [28]:
#List of all Food Groups
food_groups = list(df['Food Group'].unique())

#list of all food items
food_items = list(df['Food Item'].unique())

food_groups

['Whole Fruits',
 'Canned Fruits',
 'Dried Fruits',
 'Vegetables',
 'Potatoes',
 'Cream Products',
 'Ice Creams',
 'Yogurt Products',
 'Cheese Products',
 'Eggs',
 'Dairy Spreads',
 'Milk ',
 'Breads',
 'Pastas, Rice & Grains',
 'Meat & Meat Alternatives',
 'Fish',
 'Nuts and Seeds',
 'Cakes, Candies, Cookies & Pudding Snacks',
 'Crackers, Chips & Miscellaneous',
 'Beverages',
 'Dairy Beverages',
 'Alcoholic Beverages',
 'Spreads, Sauces & Toppings',
 'Ingredients',
 'Fast Food Items or Meals',
 'Soups',
 'Breakfast Items',
 "Kellogg's",
 'Post Cereals',
 'General Mills',
 'Quaker',
 'Other Cereal Brands']

In [12]:
# Harvard Oxalate Value Categories

# Little or no oxalate = 0 - 1 mg
#
# Low Oxalate = 2 - 4 mg
#
# Moderate Oxalate = 5 - 9 mg
#
# High Oxalate = 10 - 12 mg
#
# Very High Oxalate = 12 + mg

In [13]:
def food_groups_by_mean_oxalate_value(min_mean, max_mean):
    for group in food_groups:
        df_group = df[df['Food Group'] == group]
        mean = df_group['Oxalate Value'].mean()
        if min_mean <= mean and mean < max_mean:
            print(group)

In [45]:
#Food Groups with Very High Oxalates
food_groups_by_mean_oxalate_value(12, 999)

Vegetables
Potatoes
Pastas, Rice & Grains
Nuts and Seeds
Fast Food Items or Meals
Soups
Kellogg's
Post Cereals
Other Cereal Brands


In [46]:
#Food Groups with High Oxalates
food_groups_by_mean_oxalate_value(9, 12)

Dried Fruits
Ingredients
Breakfast Items
General Mills


In [47]:
#Food Groups with Moderate Oxalates
food_groups_by_mean_oxalate_value(4, 9)

Whole Fruits
Canned Fruits
Breads
Cakes, Candies, Cookies & Pudding Snacks
Crackers, Chips & Miscellaneous
Beverages
Spreads, Sauces & Toppings
Quaker


In [48]:
#Food Groups with Low Oxalates
food_groups_by_mean_oxalate_value(1, 4)

Yogurt Products
Milk 
Meat & Meat Alternatives
Dairy Beverages
Alcoholic Beverages


In [49]:
#Food Groups with Little or no Oxalate
food_groups_by_mean_oxalate_value(0, 1)

Cream Products
Ice Creams
Cheese Products
Eggs
Dairy Spreads
Fish


# Interquartiles and finding outliers

In [19]:
#First Quartile
q1 = df['Oxalate Value'].quantile(.25)

#Median
q2 = df['Oxalate Value'].median()

#Third Quartile
q3 = df['Oxalate Value'].quantile(.75)

#Interquartile Range
iqr = q3 - q1

outlier_threshold = q3 + 1.5 * iqr
extreme_outlier_threshold = q3 + 3 * iqr

print(q1, q2, q3, outlier_threshold, extreme_outlier_threshold)

1.0 4.0 13.0 31.0 49.0


In [20]:
#Very High Oxalates
upper_quarter = df[df['Oxalate Value'] >= q3]

In [21]:
#Upper Quarter excluding outliers
non_outliers = upper_quarter[upper_quarter['Oxalate Value'] < outlier_threshold]
non_outliers

,Food Group,Food Item,Serving size,OxalateCategory,Oxalate Value
2,Whole Fruits,Avocados,1 fruit,Very High,19
3,Whole Fruits,Dates,1 date,Very High,24
5,Whole Fruits,Kiwi,1 fruit,Very High,16
6,Whole Fruits,Orange,1 fruit,Very High,29
38,Canned Fruits,Canned Pineapple,1/2 cup,Very High,24
48,Dried Fruits,Dried Figs,5 pieces/fruits,Very High,24
49,Dried Fruits,Dried Pineapples,1/2 cup,Very High,30
60,Vegetables,Fava Beans,1/2 cup,Very High,20
63,Vegetables,Olives,approx 10 olives,Very High,18
64,Vegetables,Parsnip,1/2 cup,Very High,15


In [22]:
#Mild Outliers
mild_outliers = upper_quarter[upper_quarter['Oxalate Value'] >= outlier_threshold]
mild_outliers = mild_outliers[mild_outliers['Oxalate Value'] < extreme_outlier_threshold]
mild_outliers

,Food Group,Food Item,Serving size,OxalateCategory,Oxalate Value
7,Whole Fruits,Raspberries,1 cup,Very High,48
58,Vegetables,Bamboo Shoots,1 cup,Very High,35
68,Vegetables,Rutabaga,1/2 cup mashed,Very High,31
73,Vegetables,Yams,"1/2 cup, cubed",Very High,40
211,"Pastas, Rice & Grains",Miso,1 cup,Very High,40
286,Nuts and Seeds,Candies with Nuts (ex Snickers),2 oz,Very High,38
290,Nuts and Seeds,Mixed Nuts (with Peanuts),1 oz,Very High,39
293,Nuts and Seeds,Walnuts,1 cup or 7 nuts,Very High,31
300,"Cakes, Candies, Cookies & Pudding Snacks",Brownies,1 oz or 1/2 brownie,Very High,31
303,"Cakes, Candies, Cookies & Pudding Snacks",Candies with Nuts (ex Snickers),2 oz,Very High,38


In [23]:
#Extreme Outliers 
extreme_outliers = upper_quarter[upper_quarter['Oxalate Value'] >= extreme_outlier_threshold]
extreme_outliers

,Food Group,Food Item,Serving size,OxalateCategory,Oxalate Value
59,Vegetables,Beets,1/2 cup,Very High,76
61,Vegetables,Navy Beans,1/2 cup,Very High,76
62,Vegetables,Okra,1/2 cup,Very High,57
67,Vegetables,Rhubarb,1/2 cup,Very High,541
69,Vegetables,"Spinach, cooked",1/2 cup,Very High,755
70,Vegetables,"Spinach, raw",1 cup,Very High,656
119,Potatoes,French Fries (homemade or fast food),4 oz or 1/2 cup,Very High,51
120,Potatoes,Baked Potato with Skin,1 medium,Very High,97
203,"Pastas, Rice & Grains",Brown Rice Flour,1 cup,Very High,65
204,"Pastas, Rice & Grains",Buckwheat Groats,1 cup cooked,Very High,133


# Export Data

In [36]:
exported_data = df.copy()

In [37]:
def change_category(row):
    value = row['Oxalate Value']
    
    if value >= extreme_outlier_threshold:
        return "Super Fucking High"
    
    elif value >= outlier_threshold:
        return "Super High"
    
    else:
        return row['OxalateCategory']
        
    

In [39]:
exported_data['OxalateCategory'] = exported_data.apply(change_category, axis=1)

In [40]:
exported_data.to_csv("output.csv")